In [2]:
from datetime import datetime
import datetime as dt
import pandas as pd
import numpy as np
import urllib.request, json
import scipy.stats as stats
import statsmodels.api as sm

from metloom.pointdata import MesowestPointData, SnotelPointData
import dataretrieval.nwis as nwis
import pyet
import metpy.calc as calc
from metpy.units import units

In [3]:
def calc_net_lw_radiation(temperature, relative_humidity, elevation, dew_pt_temp=None):
    """Compute net longwave clear sky radiation from temperature measurements, relative humidy values and site elevation


    Args:
        temperature (array-like): mean daily temperatures (C)
        relative_humidity (array-like): mean relative humidity (%)
        elevation (float): site elevation (m)
        dew_pt_temp (array-like, optional): Dew point temperature, include if measured. Defaults to None.

    Returns:
        Series: net longwave radiations in W/m2
    """
    SIGMA = 5.67e-8
    CONSTANT = 40

    # Convert temperature to Kelvin
    temperature_K = temperature.to_xarray().squeeze()+273.15
    temperature_K.attrs = {'units':'K'}


    # Using Marks & Dozier, 1979 to calculate emissivity 
    T_prime = temperature_K + (0.0065 * elevation)
    T_prime.attrs = {'units':'K'}
    e_prime = relative_humidity*calc.saturation_vapor_pressure(T_prime)/100
    
    # Hydrostatic pressure
    standard_p = 101325*((293+(elevation*-0.0065))/293)**(-9.81*0.0299/(-.0065*8.314))
    # Emissivity estimate
    epsilon_atmos=(1.24 * (e_prime/T_prime)**(1/7) * (standard_p)/101300)

    # Longwave in estimate
    lw_in = epsilon_atmos * SIGMA * temperature_K**4 - CONSTANT

    # Calculate longwave out
    if dew_pt_temp is None:
        dew_pt_temp = calc.dewpoint_from_relative_humidity(temperature_K,(relative_humidity/100).to_xarray()).values
    lw_out = SIGMA*(dew_pt_temp+273.15)**4

    # Compute net longwave radiation
    rnet = lw_in - lw_out
    return rnet

def normalize_data(df):
    '''Normalized a pandas series'''
    norm = (df - df.mean())/df.std()
    norm.index = norm.index.astype(int)
    return norm

What I want is a figure to compare the peak swe with discharge (normalized)
- then I will have one legend for signal type (warm year, cold year, dry year, wet year, soil moisture is high or low)
- normalized sublimation estimate will be the size of each circle

## Taylor Park Met Data 1988-2021

In [4]:
# Get data and filter appropriately
taylor_park_df = pd.read_csv('data/taylor_park_daily_met.txt',sep='\s+')
taylor_park_elev = 3201
taylor_park_df = taylor_park_df.set_index('Date')
taylor_park_df = taylor_park_df.replace(-9999,np.nan)
taylor_park_df.index = pd.to_datetime(taylor_park_df.index)
taylor_park_df['Year'] =  taylor_park_df.index.year.where(taylor_park_df.index.month < 10, taylor_park_df.index.year + 1)
taylor_park_df = taylor_park_df.rename(columns={'Year':'Water_Yr'})

# Calculated lw net and winterize the data
winter_taylor_park_df = taylor_park_df.loc[(taylor_park_df.index.month < 5) | (taylor_park_df.index.month == 12) ]
net_taylor_winter = calc_net_lw_radiation(temperature=winter_taylor_park_df['AveT'], 
                      relative_humidity=winter_taylor_park_df['AveRH'],
                      elevation=taylor_park_elev
                        )


In [22]:
# Years in highest and lowest quartile of temperature:
taylor_park_annual_df = taylor_park_df[(taylor_park_df['Water_Yr']>1988) &(taylor_park_df['Water_Yr']<2022)].groupby(['Water_Yr']).mean()

warm_years = taylor_park_annual_df[taylor_park_annual_df['AveT']>np.percentile(taylor_park_annual_df['AveT'],75)].index
cold_years = taylor_park_annual_df[taylor_park_annual_df['AveT']<np.percentile(taylor_park_annual_df['AveT'],25)].index

In [23]:
sublimation_taylor_pm = pyet.pm(tmean=winter_taylor_park_df['AveT'],
        tmax=winter_taylor_park_df['MaxT'],
        tmin=winter_taylor_park_df['MinT'],
        rhmax=winter_taylor_park_df['MaxRH'],
        rhmin=winter_taylor_park_df['MinRH'],
        wind=winter_taylor_park_df['WSpd'],
        rh=winter_taylor_park_df['AveRH'],
        rn=net_taylor_winter*3600*24/1e6 ,
        elevation=taylor_park_elev,
        r_l=0,
        r_s=0,
        a_sh=0.5,
        ra_method=1)
sublimation_taylor_pm_df = sublimation_taylor_pm.to_frame(name='sublimation_mm')
sublimation_taylor_pm_df['water_year'] =  sublimation_taylor_pm_df.index.year.where(sublimation_taylor_pm_df.index.month < 10, sublimation_taylor_pm_df.index.year + 1)
sublimation_taylor_totals = sublimation_taylor_pm_df['sublimation_mm'].groupby(sublimation_taylor_pm_df['water_year']).sum()
sublimation_taylor_totals.rename = 'sublimation'  
      

## Get Snotel or Snow Course Data
Compute estimated sublimation at Taylor Park
Pull daily snotel point data from 1991-2022 from Butte snotel station

In [24]:

snotel_point = SnotelPointData("380:CO:SNTL", "Butte")
butte_df = snotel_point.get_daily_data(
    datetime(1991, 1, 1), datetime(2021, 9, 30),
    [snotel_point.ALLOWED_VARIABLES.PRECIPITATION,
     snotel_point.ALLOWED_VARIABLES.PRECIPITATIONACCUM,
     snotel_point.ALLOWED_VARIABLES.SNOWDEPTH,
     snotel_point.ALLOWED_VARIABLES.SWE,
     snotel_point.ALLOWED_VARIABLES.TEMP]
)
butte_df = butte_df.droplevel(1)

# Add water year
butte_df['water_year'] =  butte_df.index.year.where(butte_df.index.month < 10, butte_df.index.year + 1)

# Get max swe
butte_max_swe_mm = butte_df['SWE'].groupby(butte_df.index.year).max()*25.4

In [25]:
# Wet years and dry years (top and bottom quartile)
butte_precip_annual = butte_df['ACCUMULATED PRECIPITATION'].groupby(butte_df['water_year']).max()

wet_years = butte_precip_annual[butte_precip_annual>np.percentile(butte_precip_annual,75)].index
dry_years = butte_precip_annual[butte_precip_annual<np.percentile(butte_precip_annual,25)].index

In [26]:
summer_precip = butte_df.loc[(butte_df.index.month>5) & (butte_df.index.month<10)]['PRECIPITATON'].groupby(butte_df.loc[(butte_df.index.month>5) & (butte_df.index.month<10)].index.year).sum()
tmp = []
yr = []
for i,val in enumerate(summer_precip):
    if i < len(summer_precip)-1:
        tmp.append(np.mean([val,summer_precip.iloc[i+1]]))
        yr.append(summer_precip.index[i]+2)

previous_2yr_anomaly = pd.Series(data=tmp,index=yr)
previous_2yr_anomaly = normalize_data(previous_2yr_anomaly)

wet_soil_years = previous_2yr_anomaly[previous_2yr_anomaly>np.percentile(previous_2yr_anomaly,75)].index
dry_soil_years = previous_2yr_anomaly[previous_2yr_anomaly<np.percentile(previous_2yr_anomaly,25)].index

## Get river data    

In [27]:
def get_mean_discharge(site, spring_only=False):
    monthly_discharge_stats = nwis.get_stats(sites=site, statReportType='monthly', statTypeCd='mean')[0]
    monthly_discharge_stats = monthly_discharge_stats[monthly_discharge_stats['parameter_cd']==60]

    # Compute discharge means
    # If only spring is wanted, filter out other months
    if spring_only:
        monthly_discharge_stats = monthly_discharge_stats[monthly_discharge_stats['month_nu'].isin([5,6,7])]
    discharge_means = monthly_discharge_stats['mean_va'].groupby(monthly_discharge_stats['year_nu']).mean()
    return discharge_means

In [28]:
east_river_annual_discharge = get_mean_discharge('09112500', spring_only=True)

In [29]:
df = pd.concat([normalize_data(east_river_annual_discharge.loc[1991:]), normalize_data(butte_max_swe_mm),normalize_data(sublimation_taylor_totals)],axis=1)
df['precip_code'] = None
df.loc[dry_years,'precip_code']='dry'
df.loc[wet_years,'precip_code']='wet'

df['2yr_anom_code'] = None
df.loc[dry_soil_years,'2yr_anom_code']='dry'
df.loc[wet_soil_years,'2yr_anom_code']='wet'

df['temp_code'] = None
df.loc[warm_years,'temp_code']='warm'
df.loc[cold_years,'temp_code']='cold'

df['water_year'] = df.index.astype('str')


In [30]:
import altair as alt

scale = alt.Scale(domain=['warm', None, 'cold'],
                  range=['red', 'grey', 'blue'])

plot1 = alt.Chart(df).mark_point().encode(
    alt.X('SWE', title='Annual Maximum SWE at Butte (Normalized)'),
    alt.Y('mean_va', title='Spring Mean Discharge (Normalized)'),
    alt.Color('temp_code',scale=scale),
    alt.Size('sublimation_mm', scale=alt.Scale(domain=[-3,2]))
    )
plot_precip = alt.Chart(df).mark_point().encode(
    alt.X('SWE', title='Annual Maximum SWE at Butte (Normalized)'),
    alt.Y('mean_va', title='Spring Mean Discharge (Normalized)'),
    alt.Color('precip_code', scale=scale)
    )

plot2 =alt.Chart(pd.DataFrame({
    'x': [-2, 2.5],
    'y': [-2, 2.5],
})
).mark_line(color='grey').encode(
    x= 'x',
    y= 'y',
    # label='1 to 1 line'
)

plot1 + plot2

alt.LayerChart(...)

In [21]:
plot1 = alt.Chart(df).mark_point().encode(
    alt.X('SWE', title='Annual Maximum SWE at Butte (Normalized)'),
    alt.Y('mean_va', title='Spring Mean Discharge (Normalized)'),
    alt.Color('temp_code',scale=scale),
    alt.Size('sublimation_mm', scale=alt.Scale(domain=[-3,2]))
    )

array([1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])